<a href="https://colab.research.google.com/github/GabrielGamaUFS/Evolucao_Software_2025-2_DeepResearch_atividade2/blob/main/ESII_atv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalar as bibliotecas necessárias

!pip install transformers accelerate torch bitsandbytes

In [ ]:
import os

# Define o diretório de destino no Colab
repo_dir = "/content/DeepResearch"

# Verifica se a pasta já existe antes de clonar
if not os.path.exists(repo_dir):
    print(f"A clonar https://github.com/Alibaba-NLP/DeepResearch para {repo_dir}...")

    !git clone https://github.com/Alibaba-NLP/DeepResearch.git
    print("Repositório clonado com sucesso.")
else:
    print(f"Repositório já existe em {repo_dir}.")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
                                                 # Reinicie a sessão sempre que trocar o modelo
# Nome do modelo que você quer
model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"  # <--- Alterar pelo modelo escolhido:
                                                 # deepseek-ai/deepseek-coder-6.7b-instruct
print(f"Carregando {model_id} em 4-bit...")      # codellama/CodeLlama-7b-Instruct-hf
                                                 # mistralai/Mistral-7B-Instruct-v0.3
# --- Configuração de 4-bit  ---                 # microsoft/Phi-3-mini-128k-instruct
bnb_config = BitsAndBytesConfig(                 # Qwen/Qwen2.5-7B-Instruct
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Carregar o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Carregar o modelo aplicando a configuração de 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # <-- Aplicando a configuração de 4-bit
    device_map="auto"                # "auto" coloca o modelo na GPU
)

print("----------------------------------------------------------")
print(f"Modelo {model_id} carregado com sucesso em 4-bit!")
print("----------------------------------------------------------")

In [ ]:
import subprocess
import glob

# --- 1. FUNÇÃO PARA EXTRAIR INFORMAÇÕES DO REPOSITÓRIO ---
def get_repo_context(repo_path):
    context_data = ""

    # A. Listar Branches e Tags (Indica versionamento)
    try:
        branches = subprocess.check_output(["git", "-C", repo_path, "branch", "-r"], text=True)
        tags = subprocess.check_output(["git", "-C", repo_path, "tag"], text=True)
        context_data += f"--- BRANCHES REMOTAS ---\n{branches}\n"
        context_data += f"--- TAGS (VERSÕES) ---\n{tags}\n"
    except Exception as e:
        context_data += f"Erro ao ler git info: {e}\n"

    # B. Ler os últimos commits (Indica padrão de commit e merge)
    try:
        # Pega os últimos 20 commits formatados para mostrar merges
        logs = subprocess.check_output(
            ["git", "-C", repo_path, "log", "--graph", "--oneline", "-n", "20"],
            text=True
        )
        context_data += f"--- HISTÓRICO DE COMMITS (GRÁFICO) ---\n{logs}\n"
    except:
        pass

    # C. Verificar Workflows do GitHub (Indica CI/CD e Release Automatizada)
    workflows = glob.glob(f"{repo_path}/.github/workflows/*.yml") + glob.glob(f"{repo_path}/.github/workflows/*.yaml")
    if workflows:
        context_data += "--- ARQUIVOS DE WORKFLOW (CI/CD) ENCONTRADOS ---\n"
        for wf in workflows:
            filename = os.path.basename(wf)
            context_data += f"Nome do arquivo: {filename}\n"
            # Lê o conteúdo dos workflows para entender o que eles fazem (ex: publish release)
            with open(wf, 'r') as f:
                context_data += f"Conteúdo de {filename}:\n{f.read()}\n\n"
    else:
        context_data += "--- NENHUM WORKFLOW DE CI/CD ENCONTRADO ---\n"

    # D. Ler README e CONTRIBUTING (Busca regras escritas)
    for doc in ["README.md", "CONTRIBUTING.md", "RELEASE.md"]:
        path = os.path.join(repo_path, doc)
        if os.path.exists(path):
            with open(path, 'r') as f:
                content = f.read()
                # Trunca se for muito grande para não estourar o contexto
                context_data += f"--- ARQUIVO {doc} ---\n{content[:2000]}...\n\n"

    return context_data

# --- 2. PREPARAR O PROMPT ---
print("Coletando dados do repositório...")
repo_context = get_repo_context(repo_dir)

# Ajustamos o texto para ser mais diretivo e evitar que o modelo se alongue demais
instrucao_tarefa = """Você é um Auditor de Código Sênior e Especialista em DevOps.
Sua tarefa é analisar os dados brutos de um repositório Git fornecidos abaixo e extrair fatos reais.
Seja conciso e objetivo em cada ponto."""

corpo_dados = f"""Aqui estão os dados extraídos do repositório:
{repo_context}

Por favor, gere a análise detalhada seguindo EXATAMENTE o formato abaixo."""

# Note que terminamos o prompt com o título do relatório para "puxar" a resposta do modelo
prompt = f"""### Instruction:
{instrucao_tarefa}

{corpo_dados}

### FORMATO DE RESPOSTA ESPERADO:
## Relatório: DeepResearch

**1. Modelo de Fluxo de Trabalho:**
* Veredito:
* Justificativa:

**2. Estratégia de Releases:**
* Veredito:
* Justificativa:

**3. Resumo Geral:**

### Response:
## Relatório: DeepResearch"""

# --- EXECUTAR A INFERÊNCIA ---
print("Gerando análise...")

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **input_ids,
    max_new_tokens=1024,   # Limite da resposta
    temperature=0.2,       # Baixa para manter o foco técnico
    repetition_penalty=1.1, # Evita que ele repita as instruções do prompt
    do_sample=True,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

# Decodificando
response = tokenizer.decode(outputs[0][input_ids.input_ids.shape[-1]:], skip_special_tokens=True)

# Exibimos o título manualmente já que o usamos para induzir a resposta
print("\n" + "="*50)
print("## Relatório: DeepResearch" + response)
print("="*50)